In [78]:
'''
    Author - Oyesh Mann Singh
    Date - 11/30/2018
    Description 
        - Usage of Word Embedding
        - Deep Learning
'''

import os
import sys
import time
import re
import string
import unicodecsv as csv
import unicodedata as un
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from gensim.models import Word2Vec, KeyedVectors

In [2]:
'''
    Read all files into one corpus file
    Break down each sentence into new line
    Remove punctuation except selective punctuation
    Lowercase
    Replace numbers with <num>
    Do not remove stop words
    Remove '.'
'''
print()

### DO NOT RUN THIS SECTION TWICE

In [88]:
'''
    DO NOT RUN
'''
in_path = './data/aclImdb/data/'
out_file = './data/raw_corpus.txt'

table = dict.fromkeys(i for i in range(sys.maxunicode) 
                        if un.category(chr(i)).startswith(('P','N','S','Cf','Cn','Cc'))
                        and i != 45 and i != 46)

# Create a corpus and remove html break 
with open(out_file, 'w', encoding='utf8') as out_f:
    for root, dirs, files in os.walk(in_path, topdown=True):
        if(len(files) > 0):
            for each_file in files:
                file_path = os.path.join(root, each_file)
                fp = open(file_path, encoding='utf-8-sig').read().replace("<br />","")
                
                fp = fp.translate(table)
                
                fp = re.sub(r"(?<!\w)[-]|[-](?!\w)",'',fp)
                
                out_f.write(fp)

out_f.close()    


In [89]:
out_file = './data/raw_corpus.txt'
in_f = open(out_file, encoding='utf-8').read()

In [117]:
from nltk.corpus import gutenberg
 
print(dir(gutenberg))
print(gutenberg.fileids())
 
text = ""
for file_id in gutenberg.fileids():
    text += gutenberg.raw(file_id)

['_LazyCorpusLoader__args', '_LazyCorpusLoader__kwargs', '_LazyCorpusLoader__load', '_LazyCorpusLoader__name', '_LazyCorpusLoader__reader_cls', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '__weakref__', '_unload', 'subdir', 'unicode_repr']
['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [118]:
from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(text)
tokenizer = PunktSentenceTokenizer(trainer.get_params())

In [119]:
# Test the tokenizer on a piece of text
sentences = "Mr. James Ph.D. someone told me Dr. Brown is not available today. I will try tomorrow."
print(tokenizer.tokenize(sentences))

['Mr. James Ph.D.', 'someone told me Dr.', 'Brown is not available today.', 'I will try tomorrow.']


In [120]:
# ['Mr. James told me Dr.', 'Brown is not available today.', 'I will try tomorrow.']
# View the learned abbreviations
print(tokenizer._params.abbrev_types)
# set([...])
# Here's how to debug every split decision
for decision in tokenizer.debug_decisions(sentences):
    pprint(decision)
    print('=' * 30)

{'tit', 'gent', 'deci', 'mur', "'dr", 'o.r', 'br', 'calp', 'len', 'oct', 'ophe', 'qu', 'cly', 'syw', 'xx', 'xxx', 'p.s', 'cass', 'j', 'var', 'ro', 'mss', 'n.e', 'cin', 'a.s', 'ara', 'treb', 'macb', 'amb', 'ant', 'pol', 'brut', 'guil', 'gen', 'malc', 'p.h', 'cop', 'cob', 'm', 'ave', 'cyn', 'w', 'm.d', 'vol', 'ment', 'pind', "'mr", 'jun', 'talb', 'n', 'k.c', 'ult', 'finsb', 'moo', 's.w.f', 'ed', 'g.k', 'ang', 'hora', 'sw', 'fla', 'clo', 'luc', 'cas', 'ros', 'boa', 'por', 'polon', 'b', 'osr', 'lep', 'g', 'v.e', 'wm', 'k', 'vat', 'm.p', 'u', 'feb', 'l10', 'p', 'esq', 'mes', 'ser', 'ami', 'lbs', 'dard', 'hag', 'rosin', 'fred', 'cath', 'cai', 'gho', 'fran', 'etc', 'poe', 'bap', 'rev', 'ple', 'macd', 'trans', "'w", 'prima', 'phil', 'dec', 'i.e', 'laer', 'mal', 'doct', 'messa', 'p.m', 'cassi', 'sey', 'c', 'burs', 'clit', 'mt', '1.ple', 'mac', 'ely', 'dut', 'bru', 'cic', 'hon', 'y.sey', 'f', 'banq', 'ibid', 'u.s', 'foh', 'stra', 'a.d', 'hec', 'dag', 'ger', 'octa'}
{'break_decision': False,
 'co

In [125]:
tokenizer._params.abbrev_types.add('d')
 
print(tokenizer.tokenize(sentences))
# ['Mr. James told me Dr. Brown is not available today.', 'I will try tomorrow.']
 
for decision in tokenizer.debug_decisions(sentences):
    pprint(decision)
#     print('=' * 30)

['Mr. James Ph.D.', 'someone told me Dr. Brown is not available today.', 'I will try tomorrow.']
{'break_decision': False,
 'collocation': True,
 'period_index': 2,
 'reason': 'known collocation (both words)',
 'text': 'Mr. James',
 'type1': 'mr.',
 'type1_in_abbrs': False,
 'type1_is_initial': False,
 'type2': 'james',
 'type2_is_sent_starter': False,
 'type2_ortho_contexts': {'BEG-UC', 'UNK-UC', 'MID-UC'},
 'type2_ortho_heuristic': 'unknown'}
{'break_decision': True,
 'collocation': False,
 'period_index': 14,
 'reason': 'default decision',
 'text': 'Ph.D. someone',
 'type1': 'ph.d.',
 'type1_in_abbrs': False,
 'type1_is_initial': False,
 'type2': 'someone',
 'type2_is_sent_starter': False,
 'type2_ortho_contexts': {'UNK-LC', 'BEG-UC', 'MID-LC'},
 'type2_ortho_heuristic': False}
{'break_decision': None,
 'collocation': False,
 'period_index': 34,
 'reason': 'default decision',
 'text': 'Dr. Brown',
 'type1': 'dr.',
 'type1_in_abbrs': True,
 'type1_is_initial': False,
 'type2': 'brown

In [102]:
# -*- coding: utf-8 -*-
# https://stackoverflow.com/a/31505798/4595807
'''
    Split reviews into sentences with special conditions
'''
import re
alphabets= "([A-Za-z])"
numbers = "([0-9])"
# symbols = "[,\\\/@#$&%\-'\(\):;=\+*\^\~\[\]\"\|_`><¿”’\´½‘£ß]"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(numbers," <num> ",text)
#     text = re.sub(symbols," ",text)
    
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".","<stop>")
    text = text.replace("?","<stop>")
    text = text.replace("!","<stop>")
    text = text.replace("<prd>",".")
    text = text.replace("<br />","")
    
    text = text.lower()
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences if len(s) > 1]
    return sentences

In [82]:
sent = split_into_sentences(in_f)

In [83]:
print(sent[85669])

at least lucas didnt recycle his old footage


In [84]:
print(in_f[1:100])

nce again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrifi


In [85]:
'''
    Tokenize each sentence
'''
import nltk
total_sents = []

for each in sent:
    total_sents.append(each.split())

In [86]:
total_sents[0:5]

['yes',
 'it',
 'might',
 'be',
 'not',
 'historically',
 'accurateactually',
 'only',
 'soldiers',
 'of',
 'th',
 'rota',
 'were',
 'killed',
 'there',
 'and',
 'yes',
 'it',
 'has',
 'some',
 'mistakes',
 'and',
 'exaggerationbended',
 'machine',
 'gun',
 'come',
 'on',
 'or',
 'the',
 'that',
 'history',
 'lesson',
 'about',
 'how',
 'afghanistan',
 'was',
 'never',
 'conquered',
 'by',
 'anyone',
 'educated',
 'russian',
 'officer',
 'would',
 'know',
 'history',
 'much',
 'better',
 'than',
 'that',
 'take',
 'for',
 'example',
 'british',
 'campaign',
 'in',
 'afghanistan']

In [87]:
'''
    Creating word2vec embeddings
'''
from gensim.test.utils import get_tmpfile
from gensim.models import Word2Vec

path = get_tmpfile("./data/word2vec.model")

model = Word2Vec(total_sents, size=50, window=5, min_count=1, workers=4)
model.wv.save_word2vec_format('./data/word2vec.txt')